# QML.jl: Cross-platform GUIs for Julia

<br />
### JuliaCon 2017
<br />
### Bart Janssens
### Royal Military Academy, Belgium

# Introduction

## QML.jl Background

* Question on julia-users mailing list
* Started as a test of CxxWrap.jl
* Mostly implemented in C++ (300 lines Julia, 1700 lines C++)
* Pretty useful option for cross-platform GUI now


## Why QML?
* Cross-platform, native look
* Forces separation between GUI and backend code
* Specific language adapted to writing a GUI
* Support for visual designer tools
* Large community

## Overview
* QML basics
* Basic communication between Julia and QML
* Using a ListModel
* Embedding a Display
* OpenGL integration
* GR integration
* Future work

# QML basics
* JavaScript-based declarative language for describing a GUI
* Part of Qt, but "separate" from the C++ widgets
* Supports embedding JavaScript
* Connect JavaScript, C++ and now Julia functions to signals (e.g. button clicks)
* The backend (Julia here) does not call into the GUI
* Context properties allow "injecting" data


```javascript
import QtQuick 2.0
import QtQuick.Controls 1.0
import QtQuick.Layouts 1.0

ApplicationWindow {
  title: "Hello"
  visible: true

  ColumnLayout {
    Text { text: "Hello world!" }

    Button {
      text: "Goodbye world!"
      onClicked: Qt.quit()
    }
  }
}
```

## Running QML from Julia

In [1]:
using QML

In [2]:
# Create a QML application using a path to a QML file:
@qmlapp "qml/hello1.qml"
# Execute the application created by the previous call:
exec()

# Interacting with Julia
Let's call a Julia function, so we will replace
```js
Button {
  text: "Goodbye world!"
  onClicked: Qt.quit()
}
```
with
```javascript
//...
import org.julialang 1.0
//...
Button {
  text: "Goodbye world!"
  onClicked: Julia.quit()
}
```

## Expose a function to QML:

In [3]:
@qmlfunction quit

In [ ]:
# Create QML application and execute:
@qmlapp "qml/hello2.qml"
exec()

## Pass data
### Using functions
Returned values are converted to QML:

In [1]:
caption() = "Enter a value:"

caption (generic function with 1 method)

So are arguments:

In [2]:
function settext(s::String)
    global textfield_content
    textfield_content = s
    return
end;

textfield_content = ""

""

```js
ColumnLayout {
    Text { text: Julia.caption() }

    TextField {
      id: textEntry
      onTextChanged: Julia.settext(text)
    }

    Text { text: Julia.uppercase(textEntry.text) }
}
```

Execute asynchronously:

In [3]:
using QML
@qmlfunction uppercase caption settext
@qmlapp "qml/functions.qml"
exec_async()

In [4]:
# Show that the text was correctly passed to Julia
textfield_content

"Jul"

In [ ]:
quit()

### Pass data using context properties of composite types

In [1]:
mutable struct TextRef
  value::String
end

textproperty = TextRef("text");

Textproperty is a "context property", available from QML:

In [2]:
using QML
@qmlapp "qml/properties1.qml" textproperty

It can also be read from Julia, and is in fact a wrapped version of the original data:

In [3]:
# qmlcontext() is the root for all context properties
# @qmlget/set "overload" the dot
dump(@qmlget qmlcontext().textproperty)

QML.JuliaObjectRef
  cpp_object: Ptr{Void} Ptr{Void} @0x00007fcd8afc01e0


A look at the QML:
```javascript
ColumnLayout {
  Text { text: textproperty.value }

  TextField {
    onTextChanged: textproperty.value = text
  }
}
```

Run the GUI:

In [4]:
exec()

Check that the data made it to Julia:

In [5]:
println("Entered text: $(textproperty.value)")

Entered text: test


## Notification
### Notify QML from Julia using properties


In [9]:
numclicks = 0

function addclick()
  global numclicks
  numclicks += 1
  @qmlset qmlcontext().numclicks = numclicks
end

@qmlfunction addclick

```javascript
ColumnLayout {
  Text { text: numclicks }

  Button {
    text: "Click me!"
    onClicked: Julia.addclick()
  }
}
```

In [10]:
@qmlapp "qml/properties2.qml" numclicks
exec()
println("Number of clicks: $numclicks")

Number of clicks: 3


### Notify QML from Julia using signals

In [11]:
numclicks = 0

function addclick2()
  global numclicks
  numclicks += 1
  @emit clickAdded(Int32(numclicks))
end

@qmlfunction addclick2

```javascript
ColumnLayout {
  Text { id: clickDisplay; text: "0" }

  Button {
    text: "Click me!"
    onClicked: Julia.addclick2()
  }
}

JuliaSignals {
  signal clickAdded(int numclicks)
  onClickAdded: clickDisplay.text = numclicks
}
```

In [12]:
@qmlapp "qml/signals.qml"
exec()
println("Number of clicks: $numclicks")

Number of clicks: 6


### A more complete example: FizzBuzz

In [14]:
# The simulation data
mutable struct FizzBuzz
    result::String
    last::String
end

fizzbuzz = FizzBuzz("", "No FizzBuzz yet!")

# An advanced Julia processing algorithm, unaware of the GUI
function do_fizzbuzz(f::FizzBuzz, s::String)
    x = parse(Int, s)
    if x % 15 == 0
        f.result = "FizzBuzz"
        f.last = s
    elseif x % 3 == 0
        f.result = "Fizz"
    elseif x % 5 == 0
        f.result = "Buzz"
    else
        f.result = s
    end
end;

Text entry with validation:
```javascript
TextField {
  Layout.fillWidth: true
  placeholderText: "Input a number..."
  validator: IntValidator {
    bottom: 1
    top: 100
  }

  onAccepted: {
    Julia.do_fizzbuzz(fizzbuzz, text);
    fizzbuzz.update();
  }
}
```

Display the results using a context property:
```javascript
GroupBox {
  title: "Result"
  Layout.fillWidth: true
  Text { text: fizzbuzz.result }
}
GroupBox {
  title: "Last"
  Layout.fillWidth: true
  Text { text: fizzbuzz.last }
}
```

In [15]:
using QML
@qmlfunction do_fizzbuzz
@qmlapp "qml/fizzbuzz.qml" fizzbuzz
exec()

## QML states

A simple text input again:
```javascript
TextField {
  Layout.fillWidth: true
  placeholderText: "Input a number..."
  validator: IntValidator {}

  // Have Julia check for an even or odd number
  onAccepted: { 
    Julia.process_state(juliastate, text);
    juliastate.update();
  }
}
```

The state change triggers property adjustments:
```javascript
Rectangle {
  id: statusRect
  state: juliastate.state

  Text { id: statusText; anchors.centerIn: parent }

  states: [
    // ...
    State {
      name: "ODD"
      PropertyChanges { target: statusRect; color: "red"}
      PropertyChanges { target: statusText; text: qsTr("Odd") }
    },
    // ...
  ]
}
```

In [16]:
mutable struct JuliaState
    state::String
end

juliastate = JuliaState("STARTUP")

function process_state(state::JuliaState, s::String)
    (state.state = parse(Int,s) % 2 == 0 ? "EVEN" : "ODD")
end

process_state (generic function with 1 method)

Launch as usual

In [17]:
using QML
@qmlfunction process_state
@qmlapp "qml/states.qml" juliastate
exec()

# ListModel
Custom class to pass data to model-consuming QML entities (e.g. lists). Example:

In [18]:
mutable struct Fruit
  name::String
  cost::Float64
  description::String
end

fruitlist = [
  Fruit("Apple", 2.45, "Deciduous"),
  Fruit("Banana", 1.95, "Seedless"),
  Fruit("Cumquat", 3.25, "Citrus"), 
  Fruit("Durian", 9.95, "Tropical")];

Make a model out of the Julia array:

In [19]:
using QML
fruitModel = ListModel(fruitlist)

QML.ListModelAllocated(Ptr{Void} @0x00007ffbcf784cb0)

QML using the model:
```qml
TableView {
  model: fruitModel
  
  TableViewColumn { role: "name"; title: "Name" }
  TableViewColumn { role: "cost"; title: "Cost" }
  TableViewColumn { role: "description"; title: "Description" }
}
```
Run as usual, with the `fruitModel` as context property

In [20]:
@qmlapp "qml/listmodel.qml" fruitModel
exec()

## Modify using Delegate
```qml
Component {
  id: listDelegate
  RowLayout {
    Text { text: name }
    Text { text: cost }
    TextField {
      text: description
      onAccepted: description = text
    }
  }
}
```

Integrated in ListView:

In [21]:
@qmlapp "qml/listdelegate.qml" fruitModel
exec()

In [22]:
fruitlist

4-element Array{Fruit,1}:
 Fruit("Apple", 2.45, "Deciduous")
 Fruit("Banana", 1.95, "Hmmm")    
 Fruit("Cumquat", 3.25, "?")      
 Fruit("Durian", 9.95, "Tropical")

## Not limited to lists

In [23]:
# Represents the state related to a single emoji
type EmojiState
  emoji::String
  numclicks::Float64
  bgcolor::String
  ex::Float64
  ey::Float64
end

# Build a list of emoji, positioned randomly
emoji = EmojiState[]
randpos() = rand()*0.8+0.1
for (i,e) in enumerate(["😁", "😃", "😆", "😎", "😈"])
  push!(emoji, EmojiState(e,0, i%2 == 0 ? "blue" : "yellow", randpos(), randpos()))
end
emojiModel = ListModel(emoji) # passed to QML

QML.ListModelAllocated(Ptr{Void} @0x00007ffbd1fc83b0)

Use a `Repeater` with a `Rectangle` as delegate:
```qml
Repeater { // Repeat for each emoji
  anchors.fill: parent
  model: emojiModel

  Rectangle {
    color: bgcolor
    x: ex*appRoot.width
    y: ey*appRoot.height
    Text { text: emoji }
  
    MouseArea {
      anchors.fill: parent
      onClicked: numclicks += 1
      onPressed: parent.color = "white"
      onReleased: parent.color = bgcolor
    }
  }
}
```

In [24]:
@qmlapp "qml/repeater.qml" emojiModel
exec()

In [25]:
for e in emoji
  println("$(e.emoji) was clicked $(Int(e.numclicks)) times")
end

😁 was clicked 3 times
😃 was clicked 0 times
😆 was clicked 0 times
😎 was clicked 6 times
😈 was clicked 0 times


# Julia display

In [6]:
using QML
using PyPlot

function plotsin(d::JuliaDisplay, w, h)
  f = figure(figsize=(w/80-0.7,h/80-0.5))
  x = 0:π/100:2π
  plt = plot(x,sin.(x))
  display(d, f)
  close(f)
  return
end

objc[19783]: Class RunLoopModeTracker is implemented in both /usr/local/opt/qt5/lib/QtCore.framework/Versions/5/QtCore (0x11faf0e30) and /Users/bjanssens/.julia/v0.6/Conda/deps/usr/lib/libQt5Core.5.dylib (0x12edb88b8). One of the two will be used. Which one is undefined.
objc[19783]: Class NotificationReceiver is implemented in both /usr/local/opt/qt5/lib/QtWidgets.framework/Versions/5/QtWidgets (0x11eb6a0a0) and /Users/bjanssens/.julia/v0.6/Conda/deps/usr/lib/libQt5Widgets.5.dylib (0x131f8dca8). One of the two will be used. Which one is undefined.


plotsin (generic function with 1 method)

The `JuliaDisplay` is a custom QML component:
```qml
JuliaDisplay {
  id: jdisp
  Layout.fillWidth: true
  Layout.fillHeight: true
}
```
It is passed to Julia on a button click here:
```qml
Button {
  text: "Plot"
  onClicked: Julia.plotsin(jdisp, jdisp.width, jdisp.height)
}
```

In [7]:
@qmlfunction plotsin
@qmlapp "qml/plot.qml"
exec()

In [ ]:
quit()

# OpenGL and GLVisualize
## OpenGL triangle

In [1]:
# MUST disable threading in Qt
ENV["QSG_RENDER_LOOP"] = "basic"

using CxxWrap
using QML
using ModernGL, GeometryTypes, GLAbstraction

In [2]:
mutable struct Corner
  id::Int32
  cx::Float64
  cy::Float64
end

In [3]:
function render()
  # Draw a triangle. Code mostly from the tutorials in GLAbstraction.
  vao = Ref(GLuint(0))
  glGenVertexArrays(1, vao)
  glBindVertexArray(vao[])

  # The vertices of our triangle
  vertices = Point2f0[(c.cx, c.cy) for c in corners] # note Float32

  # Create the Vertex Buffer Object (VBO)
  vbo = Ref(GLuint(0))   # initial value is irrelevant, just allocate space
  glGenBuffers(1, vbo)
  glBindBuffer(GL_ARRAY_BUFFER, vbo[])
  glBufferData(GL_ARRAY_BUFFER, sizeof(vertices), vertices, GL_STATIC_DRAW)

  # The vertex shader
  vertex_source = """
  #version 150

  in vec2 position;

  void main()
  {
      gl_Position = vec4(position, 0.0, 1.0);
  }
  """

  # The fragment shader
  fragment_source = """
  # version 150

  out vec4 outColor;

  void main()
  {
      outColor = vec4(1.0, 1.0, 1.0, 1.0);
  }
  """

  # Compile the vertex shader
  vertex_shader = glCreateShader(GL_VERTEX_SHADER)
  glShaderSource(vertex_shader, Vector{UInt8}(vertex_source))  # nicer thanks to GLAbstraction
  glCompileShader(vertex_shader)
  # Check that it compiled correctly
  status = Ref(GLint(0))
  glGetShaderiv(vertex_shader, GL_COMPILE_STATUS, status)
  if status[] != GL_TRUE
      buffer = Array(UInt8, 512)
      glGetShaderInfoLog(vertex_shader, 512, C_NULL, buffer)
      error(bytestring(buffer))
  end

  # Compile the fragment shader
  fragment_shader = glCreateShader(GL_FRAGMENT_SHADER)
  glShaderSource(fragment_shader, Vector{UInt8}(fragment_source))
  glCompileShader(fragment_shader)
  # Check that it compiled correctly
  status = Ref(GLint(0))
  glGetShaderiv(fragment_shader, GL_COMPILE_STATUS, status)
  if status[] != GL_TRUE
      buffer = Array(UInt8, 512)
      glGetShaderInfoLog(fragment_shader, 512, C_NULL, buffer)
      error(bytestring(buffer))
  end

  # Connect the shaders by combining them into a program
  shader_program = glCreateProgram()
  glAttachShader(shader_program, vertex_shader)
  glAttachShader(shader_program, fragment_shader)
  glBindFragDataLocation(shader_program, 0, "outColor") # optional

  glLinkProgram(shader_program)
  glUseProgram(shader_program)

  # Link vertex data to attributes
  pos_attribute = glGetAttribLocation(shader_program, "position")
  glVertexAttribPointer(pos_attribute, length(eltype(vertices)),
                        GL_FLOAT, GL_FALSE, 0, C_NULL)
  glEnableVertexAttribArray(pos_attribute)

  # Set background
  glClearColor(0.,0.4,0.8,1.)
  glEnable(GL_DEPTH_TEST)
  glDepthFunc(GL_LESS)
  glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

  # Render triangle
  glDrawArrays(GL_TRIANGLES, 0, length(vertices))
end

render (generic function with 1 method)

In [4]:
render_triangle = CxxWrap.safe_cfunction(render, Void, ())

CxxWrap.SafeCFunction(Ptr{Void} @0x00000001141b8150, Void, Type[])

In [5]:
corners = [Corner(1,0,0.5), Corner(2,0.5,-0.5), Corner(3,-0.5,-0.5)]
cornersModel = ListModel(corners)

QML.ListModelAllocated(Ptr{Void} @0x00007fbd669b6a70)

In [8]:
@qmlapp "qml/opengl.qml" render_triangle cornersModel
exec()
corners

3-element Array{Corner,1}:
 Corner(1, 0.196859, 0.665359)  
 Corner(2, 0.794703, -0.0290156)
 Corner(3, -0.161391, -0.518094)

In [ ]:
quit()

# GR integration

In [1]:
ENV["QSG_RENDER_LOOP"] = "basic" # Disable threads
using CxxWrap # safe_cfunction
using GR
using QML

In [2]:
mutable struct Point
    id::Int32
    px::Float64
    py::Float64
end

ptrange = 1:5

points = [Point(i,rand(),rand()) for i in ptrange];

ndcx = Any[0.0 for i in ptrange]
ndcy = Any[0.0 for i in ptrange];

In [3]:
function plotline(p::QML.QPainterRef, ::QML.JuliaPaintedItemRef)
    ENV["GKSwstype"] = 381
    ENV["GKSconid"] = split(repr(p.cpp_object), "@")[2]
    
    dev = device(p)
    plt = gcf()
    plt[:size] = (width(dev), height(dev))

    GR.setwindow(0,1,0,1)
    
    global ndcx
    global ndcy
    
    plot([p.px for p in points],[p.py for p in points],xlim=(0,1),ylim=(0,1))
    
    for pt in points
        (ndcx[pt.id], ndcy[pt.id]) = GR.wctondc(pt.px, pt.py)
    end
    
    @qmlset qmlcontext().ndcx = ndcx
    @qmlset qmlcontext().ndcy = ndcy

    return
end

plotline (generic function with 1 method)

In [4]:
paint_cfunction = safe_cfunction(plotline, Void, (QML.QPainterRef,QML.JuliaPaintedItemRef))

CxxWrap.SafeCFunction(Ptr{Void} @0x000000010f92bbd0, Void, Type[QML.QPainterRef, QML.JuliaPaintedItemRef])

In [5]:
pointsModel = ListModel(points)

QML.ListModelAllocated(Ptr{Void} @0x00007ffa859f1ee0)

In [12]:
@qmlapp "qml/gr.qml" paint_cfunction pointsModel ndcx ndcy
exec()
points

5-element Array{Point,1}:
 Point(1, 0.4, 0.0823566)    
 Point(2, 0.628936, 0.318055)
 Point(3, 0.337205, 0.873409)
 Point(4, 0.433491, 0.534951)
 Point(5, 0.808547, 0.114685)

# Future work
* Improve GLVisualize.jl integration (events)
* Extend GR.jl integration to Plots.jl
* Listmodel integrations with e.g. DataFrames.jl
* Support more mime types on the display
* Signals/slots library in Julia?